In [38]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
from datetime import datetime
import datetime

In [16]:
game_logistics_df = pd.read_csv('total_game_logistics.csv')
away_stats_df = pd.read_csv('total_away_stats.csv')
home_stats_df = pd.read_csv('total_home_stats.csv')
box_score_df = pd.read_csv('total_box_score_df.csv')

In [17]:
all_player_stats_df = away_stats_df.append(home_stats_df)

all_player_stats_df.rename(columns = {'Unnamed: 0':'Name'}, inplace = True)

# delete all rows with team totals
non_players = all_player_stats_df[(all_player_stats_df['Name'] == 'Team Totals')].index
all_player_stats_df.drop(non_players , inplace=True)

# delete all rows with players who did not play
benched_players = all_player_stats_df[(all_player_stats_df['MP'] == 'Did Not Play')].index
all_player_stats_df.drop(benched_players , inplace=True)

# delete all rows with players who did not dress
out_players = all_player_stats_df[(all_player_stats_df['MP'] == 'Did Not Dress')].index
all_player_stats_df.drop(out_players , inplace=True)


In [44]:
all_player_stats_df = all_player_stats_df.fillna(0)
all_player_stats_df.sort_values(by=['PTS'], ascending=False)

,Name,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date
267,De'Aaron Fox,33:43,14.0,25.0,0.560,2.0,5.0,0.400,7.0,9.0,...,4.0,5.0,2.0,0.0,0.0,2.0,5.0,37.0,-20.0,12/19/2022
274,Shai Gilgeous-Alexander,37:30,16.0,23.0,0.696,2.0,3.0,0.667,3.0,4.0,...,0.0,1.0,7.0,2.0,0.0,0.0,0.0,37.0,-10.0,1/20/2023
57,Nikola Vučević,31:22,15.0,23.0,0.652,2.0,5.0,0.400,4.0,5.0,...,9.0,13.0,0.0,2.0,0.0,3.0,1.0,36.0,12.0,11/17/2018
236,De'Aaron Fox,32:26,15.0,23.0,0.652,3.0,5.0,0.600,3.0,6.0,...,4.0,5.0,4.0,1.0,1.0,3.0,1.0,36.0,5.0,10/22/2022
70,Tim Hardaway Jr.,35:57,9.0,20.0,0.450,5.0,10.0,0.500,9.0,10.0,...,1.0,2.0,2.0,3.0,0.0,1.0,2.0,32.0,-24.0,11/18/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Malik Monk,13:03,0.0,4.0,0.000,0.0,2.0,0.000,0.0,1.0,...,2.0,2.0,1.0,1.0,0.0,2.0,1.0,0.0,-13.0,10/22/2022
180,Justin Robinson,2:52,0.0,3.0,0.000,0.0,1.0,0.000,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-3.0,12/19/2021
188,Daniel Theis,6:10,0.0,2.0,0.000,0.0,1.0,0.000,0.0,0.0,...,1.0,2.0,0.0,0.0,1.0,1.0,2.0,0.0,-1.0,1/14/2022
174,Tre Jones,10:20,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,...,3.0,3.0,4.0,0.0,0.0,1.0,0.0,0.0,8.0,12/19/2021


In [20]:
val = all_player_stats_df['MP'].values[0]
val2 = all_player_stats_df['MP'].values[1]

datetime_object = datetime.strptime(val, '%M:%S').time()
datetime_object2 = datetime.strptime(val2, '%M:%S').time()

timeList = [val, val2]

mysum = datetime.timedelta()
for i in timeList:
    (m, s) = i.split(':')
    d = datetime.timedelta(minutes=int(m), seconds=int(s))
    mysum += d
print(str(mysum))

'28:49'

In [56]:
col_list = all_player_stats_df.columns.tolist()
col_list.pop(0)
col_list.pop(0)
col_list.pop(2)
col_list.pop(4)
col_list.pop(6)
col_list.pop(16)

total_sum_list = []

for col in col_list: 
    sum_list = []
    sum_list = all_player_stats_df.groupby('Name')[col].sum()
    total_sum_list.append(sum_list)
    
total_sum_list

[Name
 Aaron Gordon      23.0
 Aaron Holiday      4.0
 Al Horford         6.0
 Alec Burks        11.0
 Alex Len          11.0
                   ... 
 Wes Iwundu         0.0
 Wesley Johnson     7.0
 Xavier Munford     4.0
 Zach Randolph      5.0
 Zaza Pachulia      2.0
 Name: FG, Length: 243, dtype: float64,
 Name
 Aaron Gordon      41.0
 Aaron Holiday     10.0
 Al Horford        13.0
 Alec Burks        19.0
 Alex Len          18.0
                   ... 
 Wes Iwundu         2.0
 Wesley Johnson    14.0
 Xavier Munford     8.0
 Zach Randolph     15.0
 Zaza Pachulia      2.0
 Name: FGA, Length: 243, dtype: float64,
 Name
 Aaron Gordon      6.0
 Aaron Holiday     1.0
 Al Horford        0.0
 Alec Burks        2.0
 Alex Len          0.0
                  ... 
 Wes Iwundu        0.0
 Wesley Johnson    1.0
 Xavier Munford    1.0
 Zach Randolph     1.0
 Zaza Pachulia     0.0
 Name: 3P, Length: 243, dtype: float64,
 Name
 Aaron Gordon      13.0
 Aaron Holiday      1.0
 Al Horford         0.0
 A

In [60]:
total_sum_list[0][4]

11.0

1:03:35


Original time:
2021-08-22 11:02:05
changed time:
2021-08-22 12:17:05
